In [26]:
from pathlib import Path

import pandas as pd
from icecream import colorize, ic

from backend.Database import Database

from icecream import ic, colorize

import os

ic.configureOutput(outputFunction=lambda s: print(colorize(s) + "\n"))  # print debug
ic("bonjour");

ic| 'bonjour'



In [27]:
def init_excel_to_df(path="../../data/taux_remplissage_comcom_nebbiu.xlsx"):
    excel_sheet_names = (pd.ExcelFile(path)).sheet_names
    df_sheets = pd.read_excel(path, sheet_name=excel_sheet_names, header=1)

    list_result = []

    for key, value in df_sheets.items():
        zone = key.strip().capitalize()
        list_gps = []
        for i, row in value.iterrows():
            flagGPS = False
            if i == 0:
                flagGPS = True

            date = row.iloc[1]
            for j in range(2, len(row), 3):
                num_poubelle = int((j - 2) / 3)
                poubelle_nom = str(key) + "-poubelle-" + str(num_poubelle)
                if flagGPS:
                    list_gps.append(row.iloc[j])
                gps = list_gps[num_poubelle]
                remplissage = row.iloc[j + 1]
                coeff_touriste = row.iloc[j + 2]

                list_result.append(
                    [poubelle_nom, zone, coeff_touriste, gps, date, remplissage]
                )

    df_result = pd.DataFrame(
        list_result,
        columns=["nom", "zone", "coeff_touriste", "gps", "date", "remplissage"],
    )
    return df_result


df = init_excel_to_df()
df

,nom,zone,coeff_touriste,gps,date,remplissage
0,olmeta di tuda-poubelle-0,Olmeta di tuda,1,"42°36'39""N 9°21'14""E",2023-01-01,5
1,olmeta di tuda-poubelle-1,Olmeta di tuda,1,"42°36'44""N 9°21'24""E",2023-01-01,18
2,olmeta di tuda-poubelle-2,Olmeta di tuda,1,"42°36'51""N 9°21'17""E",2023-01-01,21
3,olmeta di tuda-poubelle-3,Olmeta di tuda,1,"42°36'55""N 9°21'05""E",2023-01-01,12
4,olmeta di tuda-poubelle-4,Olmeta di tuda,1,"42°37'01""N 9°21'20""E",2023-01-01,9
...,...,...,...,...,...,...
38492,san gavino di tenda-poubelle-1,San gavino di tenda,1,"42°35'54""N 9°16'02""E",2023-12-31,97
38493,san gavino di tenda-poubelle-2,San gavino di tenda,1,"42°36'05""N 9°16'16""E",2023-12-31,92
38494,san gavino di tenda-poubelle-0,San gavino di tenda,1,"42°35'53""N 9°15'53""E",2024-01-01,103
38495,san gavino di tenda-poubelle-1,San gavino di tenda,1,"42°35'54""N 9°16'02""E",2024-01-01,100


In [28]:
zone_to_pop = {
    "Olmeta di tuda": 442,
    "Rapale": 149,
    "Santu petro di tenda": 360,
    "Barbaggio": 266,
    "Sorio": 137,
    "Pieve": 118,
    "Oletta": 1_587,
    "Saint florent": 1_634,
    "Poggio oletta": 218,
    "Vallecalle": 131,
    "Rutali": 387,
    "Farinole": 197,
    "Murato": 625,
    "Patrimonio": 746,
    "Casta": 50,
    "San gavino di tenda": 68,
}

df["population"] = df["zone"].map(lambda x: zone_to_pop[x])

In [29]:
df

,nom,zone,coeff_touriste,gps,date,remplissage,population
0,olmeta di tuda-poubelle-0,Olmeta di tuda,1,"42°36'39""N 9°21'14""E",2023-01-01,5,442
1,olmeta di tuda-poubelle-1,Olmeta di tuda,1,"42°36'44""N 9°21'24""E",2023-01-01,18,442
2,olmeta di tuda-poubelle-2,Olmeta di tuda,1,"42°36'51""N 9°21'17""E",2023-01-01,21,442
3,olmeta di tuda-poubelle-3,Olmeta di tuda,1,"42°36'55""N 9°21'05""E",2023-01-01,12,442
4,olmeta di tuda-poubelle-4,Olmeta di tuda,1,"42°37'01""N 9°21'20""E",2023-01-01,9,442
...,...,...,...,...,...,...,...
38492,san gavino di tenda-poubelle-1,San gavino di tenda,1,"42°35'54""N 9°16'02""E",2023-12-31,97,68
38493,san gavino di tenda-poubelle-2,San gavino di tenda,1,"42°36'05""N 9°16'16""E",2023-12-31,92,68
38494,san gavino di tenda-poubelle-0,San gavino di tenda,1,"42°35'53""N 9°15'53""E",2024-01-01,103,68
38495,san gavino di tenda-poubelle-1,San gavino di tenda,1,"42°35'54""N 9°16'02""E",2024-01-01,100,68


In [30]:
df_y = df["remplissage"]
df_y

0          5
1         18
2         21
3         12
4          9
        ... 
38492     97
38493     92
38494    103
38495    100
38496    100
Name: remplissage, Length: 38497, dtype: int64

In [31]:
df_x = df.drop(columns=["remplissage"])
df_x

,nom,zone,coeff_touriste,gps,date,population
0,olmeta di tuda-poubelle-0,Olmeta di tuda,1,"42°36'39""N 9°21'14""E",2023-01-01,442
1,olmeta di tuda-poubelle-1,Olmeta di tuda,1,"42°36'44""N 9°21'24""E",2023-01-01,442
2,olmeta di tuda-poubelle-2,Olmeta di tuda,1,"42°36'51""N 9°21'17""E",2023-01-01,442
3,olmeta di tuda-poubelle-3,Olmeta di tuda,1,"42°36'55""N 9°21'05""E",2023-01-01,442
4,olmeta di tuda-poubelle-4,Olmeta di tuda,1,"42°37'01""N 9°21'20""E",2023-01-01,442
...,...,...,...,...,...,...
38492,san gavino di tenda-poubelle-1,San gavino di tenda,1,"42°35'54""N 9°16'02""E",2023-12-31,68
38493,san gavino di tenda-poubelle-2,San gavino di tenda,1,"42°36'05""N 9°16'16""E",2023-12-31,68
38494,san gavino di tenda-poubelle-0,San gavino di tenda,1,"42°35'53""N 9°15'53""E",2024-01-01,68
38495,san gavino di tenda-poubelle-1,San gavino di tenda,1,"42°35'54""N 9°16'02""E",2024-01-01,68


In [32]:
df_x = df_x.drop(columns=["gps", "nom"])
df_x

,zone,coeff_touriste,date,population
0,Olmeta di tuda,1,2023-01-01,442
1,Olmeta di tuda,1,2023-01-01,442
2,Olmeta di tuda,1,2023-01-01,442
3,Olmeta di tuda,1,2023-01-01,442
4,Olmeta di tuda,1,2023-01-01,442
...,...,...,...,...
38492,San gavino di tenda,1,2023-12-31,68
38493,San gavino di tenda,1,2023-12-31,68
38494,San gavino di tenda,1,2024-01-01,68
38495,San gavino di tenda,1,2024-01-01,68
